In [8]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os
from tabulate import tabulate
from typing import Literal, List
import re
sns.set_style("whitegrid")

first load data, get all years together
what is devtest???

ik doe wat ze in paper hebben gedaan (alle data van vorige jaren = train,+ train van dit jaar), maar toch is er evenveel test als train ongeveer, wat ik raar vind.) het lijkt alsof er minder train dan test data is ook (bijv 2015) -> dat is toch raar hahah

In [17]:
def get_filenames(dataset: Literal["train","test"], path="data/downloaded/") -> List[str]:
    if dataset == "train":
        r = '[a-z]+-(\d{4})'
        onlyfiles = [f for f in os.listdir("data/downloaded") if os.path.isfile(os.path.join("data/downloaded", f)) and os.path.join("data/downloaded", f)[-5] == "A"]
        return [f for f in onlyfiles if f not in ["twitter-2016dev-A.tsv", "twitter-2016devtest-A.tsv", "twitter-2016train-A.tsv"]]
    return ["twitter-2016dev-A.tsv", "twitter-2016devtest-A.tsv", "twitter-2016train-A.tsv"]

In [18]:
def concatenate_data(dataset: Literal["train","test"], path="data/downloaded/") -> None:
    # write all data from previous years to new file, as in paper
    filenames = get_filenames(dataset, path=path)

    with open(f'{path}twitter-{dataset}.tsv', 'w') as outfile:
        for fname in filenames:
            with open(path+fname) as infile:
                for line in infile:
                    outfile.write(line)

In [19]:
concatenate_data("train")
concatenate_data("test")

Load into dataframes

In [20]:
train = pd.read_csv("data/downloaded/twitter-train.tsv", sep='\t', usecols=[1,2], names=["label", "text"])
test = pd.read_csv("data/downloaded/twitter-test.tsv", sep='\t', usecols=[1,2], names=["label", "text"])
# dev = pd.read_csv("data/downloaded/twitter-2016dev-A.tsv", sep='\t', usecols=[1,2], names=["label", "text"])

print(f"train: {round(train.shape[0]/(train.shape[0]+test.shape[0])*100)}%, test: {round(test.shape[0]/(train.shape[0]+test.shape[0])*100)}%")
print(f"train: {train.shape[0]}, test: {test.shape[0]}")

train: 80%, test: 20%
train: 40297, test: 9834


class imbalances

In [21]:
train_class_counts = train.groupby('label').count().rename({"text":"train"}, axis=1)
test_class_counts = test.groupby('label').count().rename({"text":"test"}, axis=1)
# dev_class_counts = dev.groupby('label').count().rename({"text":"dev"}, axis=1)


pd.concat([train_class_counts, test_class_counts], axis=1)

,train,test
label,,
negative,6243,1566
neutral,19096,3428
positive,14958,4840


preprocessing:
remove links? 
remove @'s ?
punctuation etc? 
